<a href="https://colab.research.google.com/github/xaknight/Deathmatch_Simulator/blob/main/mlflow_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00


In [2]:
import os
import shutil
import pprint

from random import random, randint
import mlflow.sklearn
from mlflow import log_metric, log_param, log_artifacts
from sklearn.ensemble import RandomForestRegressor
from mlflow.tracking import MlflowClient
import warnings

In [3]:
warnings.filterwarnings("ignore")
print(mlflow.__version__)

2.21.3


In [4]:
# Set the tracking server to be localhost with sqlite as tracking store
local_registry = "sqlite:///mlruns.db"
print(f"Running local model registry={local_registry}")
model_name="WeatherForecastModel"
mlflow.set_tracking_uri(local_registry)
with mlflow.start_run(run_name="LOCAL_REGISTRY") as run:
    params = {"n_estimators": 3, "random_state": 0}
    sk_learn_rfr = RandomForestRegressor(params)

    # Log parameters and metrics using the MLflow API
    mlflow.log_params(params)
    log_param("param_1", randint(0, 100))
    log_metric("metric_1", random())
    log_metric("metric_2", random() + 1)
    log_metric("metric_33", random() + 2)

    # Log and register the model at the same time
    mlflow.sklearn.log_model(
                sk_model = sk_learn_rfr,
                artifact_path = "sklearn-model",
                registered_model_name="WeatherForecastModel")
    if not os.path.exists("outputs"):
        os.makedirs("outputs")
    with open("outputs/test.txt", "w") as f:
        f.write("Looks, like I logged to the local store!")
    log_artifacts("outputs")
    shutil.rmtree('outputs')
    run_id = run.info.run_uuid

Running local model registry=sqlite:///mlruns.db


2025/04/22 05:00:22 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/04/22 05:00:22 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [6]:
client = MlflowClient()
#
# transition model stage to production
#
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="production")

# Get a list of all registered models
print("List of all registered models")
print("=" * 80)
[print(pprint.pprint(dict(rm), indent=4)) for rm in client.search_registered_models()]

# Get a list of specific versions of the named models
print(f"List of Model = {model_name} and Versions")
print("=" * 80)
[pprint.pprint(dict(mv), indent=4) for mv in client.search_model_versions("name='WeatherForecastModel'")]


List of all registered models
{   'aliases': {},
    'creation_timestamp': 1745298038194,
    'description': None,
    'last_updated_timestamp': 1745298452297,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1745298038225, current_stage='Production', description=None, last_updated_timestamp=1745298452297, name='WeatherForecastModel', run_id='909a8ff8aa8d4f81b6d730a059ba2bac', run_link=None, source='/content/mlruns/0/909a8ff8aa8d4f81b6d730a059ba2bac/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>],
    'name': 'WeatherForecastModel',
    'tags': {}}
None
List of Model = WeatherForecastModel and Versions
{   'aliases': [],
    'creation_timestamp': 1745298038225,
    'current_stage': 'Production',
    'description': None,
    'last_updated_timestamp': 1745298452297,
    'name': 'WeatherForecastModel',
    'run_id': '909a8ff8aa8d4f81b6d730a059ba2bac',
    'run_link': None,
    'source': '/content/mlruns/0/909a8ff8aa

[None]

In [7]:
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --backend-store-uri sqlite:///mlruns.db --port 5000 &")# run tracking UI in the background

In [8]:
# create remote tunnel using ngrok.com to allow local port access
# borrowed from https://colab.research.google.com/github/alfozan/MLflow-GBRT-demo/blob/master/MLflow-GBRT-demo.ipynb#scrollTo=4h3bKHMYUIG6
!pip install pyngrok --quiet
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN =
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
public_url = ngrok.connect(port="5000", proto="http", options={"bind_tls": True})
print("MLflow Tracking UI:", public_url)

WARNI [pyngrok.process.ngrok] t=2025-04-22T05:08:00+0000 lvl=warn msg="invalid tunnel configuration" pg=/api/tunnels id=3daf903d9a4c28db err="yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel\n  line 1: field options not found in type config.HTTPv2Tunnel"


PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel\n  line 1: field options not found in type config.HTTPv2Tunnel"}}
